## Evaluation SSD 300 model (tflite version)

In [1]:
import seaborn as sns
import pandas as pd
import helper as hp
import numpy as np
import matplotlib.pyplot as plt
import os
import json
from imageio import imread
from mpl_toolkits.mplot3d.axes3d import Axes3D

In [2]:
# File paths
absolute_path = os.path.abspath(os.path.dirname('__file__'))

In [173]:
# Read predictions from tflite model as dictionary
with open(absolute_path + '/trade-off/a_1_e_6_f1.txt', 'r') as json_file:
    dict_pred_tflite = json.load(json_file)
    
with open(absolute_path + '/../../data-cic/' + 'via_region_data _final.json', 'r') as json_file:
    dict_json = json.load(json_file)
ground_truth_dict = hp.clean_dict(dict_json)

In [174]:
dict_pred_tflite

[{'averageTNet': 0.701,
  'averageTPosProcess': 0.0,
  'averageTScale': 0.003,
  'averageTTotal': 0.706,
  'confidence': 0.2,
  'iou': 0.45,
  'results': [{'image': 'image_30_0017.jpg',
    'predictions': [[0.98291194,
      0.5932294,
      0.039074212,
      0.128667,
      0.55516446]]},
   {'image': 'image_13_0003.jpg',
    'predictions': [[0.9976052, 0.67813647, 0.37973204, 0.15333724, 0.6010361],
     [0.9827679, 0.19572496, 0.42700183, 0.059357107, 0.20599496]]},
   {'image': 'image_40_0014.jpg',
    'predictions': [[0.84336334, 0.33403915, 0.0606789, 0.20707995, 0.8160037],
     [0.70594203, 0.48009312, 0.09960464, 0.11367607, 0.73909926]]},
   {'image': 'image_17_0001.jpg',
    'predictions': [[0.9999343,
      0.67138964,
      0.29202697,
      0.3137151,
      0.36020926]]},
   {'image': 'image_21_0005.jpg',
    'predictions': [[0.5877901,
      0.52321106,
      0.2521027,
      0.088272095,
      0.3697311]]},
   {'image': 'image_15_0008.jpg', 'predictions': []},
   {'ima

In [175]:
# Get average processing time
averageTNet = 0
averageTPosProcess = 0
averageTScale = 0
averageTTotal = 0

num_pred = len(dict_pred_tflite)

for prediction in dict_pred_tflite:
    averageTNet += prediction['averageTNet']
    averageTPosProcess += prediction['averageTPosProcess']
    averageTScale += prediction['averageTScale']
    averageTTotal += prediction['averageTTotal']
    
print('Average Scale Time: ', averageTScale / num_pred)
print('Average PosProcess Time: ', averageTPosProcess / num_pred)
print('Average Net Time: ', averageTNet / num_pred)
print('Average Total Time: ', averageTTotal / num_pred)

Average Scale Time:  0.003
Average PosProcess Time:  0.0
Average Net Time:  0.701
Average Total Time:  0.706


In [176]:
# stores performance from different iou parameter
performance_list = []

# Iterate over each prediction with different iou
for prediction in dict_pred_tflite:
    predictions_bb = []
    ground_t_bb = []
    img_names_test = []
    info_pred = []
    # Get Confidence and IOU
    confidence = prediction['confidence']
    iou = prediction['iou']
    
    info_pred.append(confidence)
    info_pred.append(iou)
    
    # Iterate over all predictions
    for predict in prediction['results']:
        img_names_test.append(predict['image']) # Add image name
        predictions_bb.append(list(map(lambda pred: pred[1:], predict['predictions']))) # Remove confidence value
                
    # Count number of predictions
    count = 0
    for p_bb in predictions_bb:
        count += len(p_bb)
    
    # Get ground truth in the same order as the predictions
    for name in img_names_test:
        # Read image to get width and height
        img = imread(absolute_path + '/../../data-cic/preprocess_data/img_test/' + name)
        
        # Get data from json file
        b_boxes = hp.get_ground_truth(ground_truth_dict[name])
        # Normilize between [0, 1]
        b_boxes = hp.normilize_boxes(b_boxes, img.shape[1], img.shape[0])
        ground_t_bb.append(b_boxes)
    
    # Calculate performance
    print('Confidence: ', confidence, 'IOU: ', iou)
    performance = list(hp.cal_performance(ground_t_bb, predictions_bb, False))
    [info_pred.append(x) for x in performance]
    info_pred.append(count)
    info_pred.append(averageTTotal)
    performance_list.append(info_pred)
    print()

Confidence:  0.2 IOU:  0.45



In [177]:
print('Tradeoff')
# Create DataFrame to display performance
columns = ['Confidence', 'IOU', 'Presicion', 'Recall', 'F1 score', 'No. boxes', 'Time']
performance_frame = pd.DataFrame(performance_list, columns=columns)
performance_frame

Tradeoff


,Confidence,IOU,Presicion,Recall,F1 score,No. boxes,Time
0,0.2,0.45,0.785215,0.653474,0.713313,80,0.706


In [106]:
# Create DataFrame to display performance
print('VGG16 20 classes fine tune with cic')
columns = ['Confidence', 'IOU', 'Presicion', 'Recall', 'F1 score', 'No. boxes', 'Time']
performance_frame = pd.DataFrame(performance_list, columns=columns)
performance_frame

VGG16 20 classes fine tune with cic


,Confidence,IOU,Presicion,Recall,F1 score,No. boxes,Time
0,0.15,0.45,0.75937,0.742251,0.750713,98,5.22


In [127]:
# Create DataFrame to display performance
print('VGG16 20 classes')
columns = ['Confidence', 'IOU', 'Presicion', 'Recall', 'F1 score', 'No. boxes', 'Time']
performance_frame = pd.DataFrame(performance_list, columns=columns)
performance_frame

VGG16 20 classes


,Confidence,IOU,Presicion,Recall,F1 score,No. boxes,Time
0,0.15,0.45,0.623552,0.659885,0.641204,117,4.417


In [7]:
# Create DataFrame to display performance
print('VGG16 2 classes')
columns = ['Confidence', 'IOU', 'Presicion', 'Recall', 'F1 score', 'No. boxes', 'Time']
performance_frame = pd.DataFrame(performance_list, columns=columns)
performance_frame

VGG16 2 classes


,Confidence,IOU,Presicion,Recall,F1 score,No. boxes,Time
0,0.2,0.45,0.716049,0.562368,0.629972,80,2.979


In [31]:
print('VGG13')
# Create DataFrame to display performance
columns = ['Confidence', 'IOU', 'Presicion', 'Recall', 'F1 score', 'No. boxes', 'Time']
performance_frame = pd.DataFrame(performance_list, columns=columns)
performance_frame

VGG13


,Confidence,IOU,Presicion,Recall,F1 score,No. boxes,Time
0,0.2,0.45,0.758249,0.621512,0.683105,80,2.507


In [36]:
print('Mobilenetv1')
# Create DataFrame to display performance
columns = ['Confidence', 'IOU', 'Presicion', 'Recall', 'F1 score', 'No. boxes', 'Time']
performance_frame = pd.DataFrame(performance_list, columns=columns)
performance_frame

Mobilenetv1


,Confidence,IOU,Presicion,Recall,F1 score,No. boxes,Time
0,0.2,0.45,0.709704,0.645243,0.67594,90,0.548


In [41]:
print('SSDLite with Mobilenetv1')
# Create DataFrame to display performance
columns = ['Confidence', 'IOU', 'Presicion', 'Recall', 'F1 score', 'No. boxes', 'Time']
performance_frame = pd.DataFrame(performance_list, columns=columns)
performance_frame

SSDLite with Mobilenetv2


,Confidence,IOU,Presicion,Recall,F1 score,No. boxes,Time
0,0.2,0.45,0.750447,0.679705,0.713327,87,0.474


In [46]:
print('SSDLite with Mobilenetv2')
# Create DataFrame to display performance
columns = ['Confidence', 'IOU', 'Presicion', 'Recall', 'F1 score', 'No. boxes', 'Time']
performance_frame = pd.DataFrame(performance_list, columns=columns)
performance_frame

SSDLite with Mobilenetv2


,Confidence,IOU,Presicion,Recall,F1 score,No. boxes,Time
0,0.2,0.45,0.784424,0.632007,0.700015,77,0.512
